In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from functools import partial
import matplotlib.pyplot as plt
print(tf.__version__)

2.4.0


In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 256
IMAGE_SIZE = [96, 96]

In [6]:
FILENAMES1 = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/" + "*.tfrecords")
FILENAMES2 = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group2/" + "*.tfrecords")
FILENAMES3 = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group3/" + "*.tfrecords")
FILENAMES4 = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group4/" + "*.tfrecords")
FILENAMES5 = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group5/" + "*.tfrecords")
FILENAMES6 = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_splitted_by_age_2000/" + "*.tfrecords")
UTKFACE = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/extrautkface/UTKFace.tfrecords")

FILENAMES = FILENAMES1 + FILENAMES2 + UTKFACE + FILENAMES3 +FILENAMES4
split_ind = int(0.77 * len(FILENAMES))
print(len(FILENAMES))
TRAINING_FILENAMES, VALID_FILENAMES = FILENAMES[:split_ind], FILENAMES[split_ind:]
TEST_FILENAMES = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/test_tf_record_alignement/test_with_resize.tfrecords")

print("Train TFRecord Files:", len(TRAINING_FILENAMES))
print(TRAINING_FILENAMES)
print("Validation TFRecord Files:", len(VALID_FILENAMES))
print(VALID_FILENAMES)
print("Test TFRecord Files:", len(TEST_FILENAMES))
print(TEST_FILENAMES)

81
Train TFRecord Files: 62
['/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part1.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part10.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part2.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part3.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part4.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part5.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part6.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part7.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part8.tfrecords', '/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group1/dataset_part9.tf

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras-vggface
!pip install keras_applications

In [8]:
def decode_image(image):
    #image = tf.image.decode_jpeg(image, channels=3)
    image = tf.io.decode_raw(image, tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image


In [9]:
def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "label_class": tf.io.FixedLenFeature([101],tf.int64),
            "label_regr_int": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        #label = tf.cast(example["label_regr_int"], tf.int64)
        label = example["label_class"]
        print(str(label))
        return image, label
    return image

In [10]:
def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [11]:
def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
from google.colab.patches import cv2_imshow
train_dataset = get_dataset(TRAINING_FILENAMES)
valid_dataset = get_dataset(VALID_FILENAMES)
test_dataset = get_dataset(TEST_FILENAMES, labeled=True)

image_batch, label_batch = next(iter(test_dataset))


def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for n in range(1):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n] / 255.0)
        plt.title(str(label_batch[n]))
        plt.axis("off")


show_batch(image_batch.numpy(), label_batch.numpy())

In [13]:
# CUSTOM METRIC

import keras.backend as K
from tensorflow.keras.metrics import mean_absolute_error

@tf.function
def mae(y_true, y_pred):
  y_true = K.cast(y_true, y_pred.dtype)

  ages_true = tf.map_fn(lambda true: K.argmax(true), y_true, dtype=tf.int64)
  ages_pred = tf.map_fn(lambda pred: K.argmax(pred), y_pred, dtype=tf.int64)

  #fn_output_signature da usare nel caso map_fn non funzioni

  return mean_absolute_error(ages_true, ages_pred)

In [26]:
#Define model VGGFace-VGG16 Classification
from keras_vggface.vggface import VGGFace
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
vgg_model = VGGFace(weights=None, include_top=False, input_shape=(96, 96, 3))

"""
# Freeze four convolution blocks
for layer in vgg_model.layers[:15]:
    layer.trainable = False

#don't train existing weights
for layer in vgg_model.layers:
  layer.trainable = False
"""
x = vgg_model.get_layer('pool5').output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout layer to reduce overfitting
x = Dense(512, activation='relu')(x)
out = Dense(101, activation= 'softmax')(x) 
transfer_model = Model(vgg_model.input, out)

"""
# Loads the weights of pretraining
checkpoint_path = '/content/drive/MyDrive/Final_Artificial_Vision/CNN/vggFace_classificatore2.h5'
transfer_model.load_weights(checkpoint_path)
"""
# Loads the weights of full model
checkpoint_path = '/content/drive/MyDrive/Final_Artificial_Vision/CNN/vggFace_classificatore_full2.h5'
transfer_model.load_weights(checkpoint_path)

# Visualize the network
for i, layer in enumerate(transfer_model.layers):
    print(i, layer.name, layer.trainable)


transfer_model.summary()

0 input_9 True
1 conv1_1 True
2 conv1_2 True
3 pool1 True
4 conv2_1 True
5 conv2_2 True
6 pool2 True
7 conv3_1 True
8 conv3_2 True
9 conv3_3 True
10 pool3 True
11 conv4_1 True
12 conv4_2 True
13 conv4_3 True
14 pool4 True
15 conv5_1 True
16 conv5_2 True
17 conv5_3 True
18 pool5 True
19 flatten_8 True
20 dense_24 True
21 dropout_8 True
22 dense_25 True
23 dense_26 True
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 96, 96, 64)        1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 96, 96, 64)        36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 48, 64)        0         
__________________

In [27]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Final_Artificial_Vision/CNN/vggFace_classificatore_full2.h5', monitor= 'val_mae', mode= 'min', save_weights_only=False, save_best_only = True, verbose= 1)

from tensorflow.keras import layers, models, Model, optimizers

transfer_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizers.Adam(learning_rate=0.0001), metrics=[tf.keras.metrics.CategoricalAccuracy(), mae])

In [ ]:
#Train the model VGGFace-VGG16 Classification
history = transfer_model.fit(train_dataset, epochs=2, validation_data=valid_dataset, callbacks=[checkpoint])

In [ ]:
#Plot the mae details
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])